# Analysieren und Bereinigen von Daten

Worauf prüfen wir?

* **Komplettheit** wir können entsprechende Einträge entfernen oder die Werte angemessen auffüllen
* **Einzigartigkeit** entfernen von Duplikaten
* **Aktualität** Zeiträume eingrenzen
* **Akuratheit** entfernen von korrupten Daten
  

In [2]:
import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
from pyspark.sql.types import *

import schemata

spark = (
    SparkSession
    .builder
    .config("spark.dynamicAllocation.enabled", "false")
    .config("spark.sql.adaptive.enabled", "false")
    .appName("analyse")
    .master("local[4]")
    .getOrCreate()
)
sc = spark.sparkContext

from IPython.display import *
display(HTML("<style>pre { white-space: pre !important; }</style>"))

spark

In [3]:
yellow_taxi_df = (
    spark
        .read
        .option("header", True)
        .schema(schemata.yellow_taxi_schema)
        .csv("YellowTaxis_202210.csv.gz")
)
yellow_taxi_df.printSchema()

root
 |-- VendorId: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



Dateframes haben eine nützliche Methode, mit der wir uns einen guten Überblick über den Wertebereich von Spalten verschaffen können.

[Api-Doc](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.describe.html#pyspark.sql.DataFrame.describe)

In [5]:
yellow_taxi_analyzed_df = yellow_taxi_df.describe("passenger_count", "trip_distance")
yellow_taxi_analyzed_df.show()

+-------+------------------+-----------------+
|summary|   passenger_count|    trip_distance|
+-------+------------------+-----------------+
|  count|           3542392|          3675412|
|   mean|1.3846934500755421|6.206976298167039|
| stddev|0.9302303297406955|640.8236808320255|
|    min|               0.0|              0.0|
|    max|               9.0|        389678.46|
+-------+------------------+-----------------+



Wir haben hier schon ein paar Hinweise auf Datenfehler:
 
 * 0 Passagiere sind wohl ein Fehler
 * 9 Gäste sind verboten und damit auch ein Fehler


## Ausfiltern nicht akkurater Daten

In [14]:
print("Vor dem Filtern: " + str(yellow_taxi_df.count()))
yellow_taxi_df = (
    yellow_taxi_df
        .where("passenger_count > 0")
        .filter(col("trip_distance") > 0.0)
)
print("Nachdem Filtern: " + str(yellow_taxi_df.count()))

Vor dem Filtern: 3421416


Nachdem Filtern: 3421416


In Spark-UI checken (Predicate Pushdown, Dataframe SQL-Tab

## Null filtern

In [15]:
print("Vor dem Filtern: " + str(yellow_taxi_df.count()))

yellow_taxi_df = yellow_taxi_df.na.drop("all") # all=all columns are null, any=with null in one column

print("Nach dem Filtern: " + str(yellow_taxi_df.count()))


Vor dem Filtern: 3421416


Nach dem Filtern: 3421416


## Nullwerte mit anderen Werten auffüllen


In [16]:
default_value_map = {'payment_type': 5, 'RateCodeId': 1}

yellow_taxi_df = yellow_taxi_df.na.fill(default_value_map)


## Duplikate entfernen

In [17]:
print("Vor der Operation: " + str(yellow_taxi_df.count()))

# Du kannst auch eine Liste von Spalten angeben, die für die Dublikatserkennung genutzt werden
yellow_taxi_without_dup_df = yellow_taxi_df.drop_duplicates()

print("Nach der Operation: " + str(yellow_taxi_without_dup_df.count()))

Vor der Operation: 3421416


Nach der Operation: 3421415


## Beschränke Zeiträume

In [12]:
# wir haben auch Timestamp-Datentypen
yellow_taxi_df.printSchema()

root
 |-- VendorId: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = false)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = false)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [18]:
print("Vor dem Einschränken: " + str(yellow_taxi_df.count()))

yellow_taxi_df = yellow_taxi_df.where("tpep_pickup_datetime >= '2022-10-01' AND tpep_dropoff_datetime < '2022-11-01'")

print ("Nach dem Einschränken: " + str(yellow_taxi_df.count()))

Vor dem Einschränken: 3421416


Nach dem Einschränken: 3421416


## Alles zusammen machen


In [19]:
default_value_map = {'payment_type': 5, 'RateCodeId': 1}

yellow_taxi_df = spark.read.option("header", True).schema(schemata.yellow_taxi_schema).csv("YellowTaxis_202210.csv.gz")

print("Vor dem Filtern: " + str(yellow_taxi_df.count()))

yellow_taxi_df = (
    yellow_taxi_df
    .where("passenger_count > 0")
    .filter(col("trip_distance") > 0.0)
    .na.drop("all")
    .na.fill(default_value_map)
    .drop_duplicates()
    .where("tpep_pickup_datetime >= '2022-10-01' AND tpep_dropoff_datetime < '2022-11-01'")
)

print("Nach dem Filtern: + " + str(yellow_taxi_df.count()))


Vor dem Filtern: 3675412


Nach dem Filtern: + 3421415


# Transformieren

## Spalten limitieren


In [10]:
yellow_taxi_df.printSchema()

root
 |-- VendorId: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = false)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = false)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [20]:
yellow_taxi_df = (
    yellow_taxi_df.select(
        "VendorID",                                    # Spaltenname als Literal
        col("passenger_count").cast(IntegerType()),    # Spalte über die col funktion und mit Cast in einen Int überführen
        column("trip_distance").alias("TripDistance"), # Auswählen und umbennen
        yellow_taxi_df.tpep_pickup_datetime,           # als Attribute des Dataframe
        "tpep_dropoff_datetime",
        "PULocationID",
        "DOLocationID",
        "RatecodeID",
        "total_amount",
        "payment_type"
        )
)
yellow_taxi_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- TripDistance: double (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- RatecodeID: double (nullable = false)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = false)



Das schränkt auch die Größe des Dataframes ein und macht die Verarbeitung so schneller.

## Spalten umbenennen

In [21]:
yellow_taxi_df = (
    yellow_taxi_df
        .withColumnRenamed("passenger_count", "PassengerCount")
        .withColumnRenamed("tpep_pickup_datetime", "PickupTime")
        .withColumnRenamed("tpep_dropoff_datetime", "DropTime")
        .withColumnRenamed("PULocationID", "PickupLocationId")
        .withColumnRenamed("DOLocationID", "DropLocationId")
        .withColumnRenamed("total_amount", "TotalAmount")
        .withColumnRenamed("payment_type", "PaymentType")
)
yellow_taxi_df.printSchema()


root
 |-- VendorID: integer (nullable = true)
 |-- PassengerCount: integer (nullable = true)
 |-- TripDistance: double (nullable = true)
 |-- PickupTime: timestamp (nullable = true)
 |-- DropTime: timestamp (nullable = true)
 |-- PickupLocationId: integer (nullable = true)
 |-- DropLocationId: integer (nullable = true)
 |-- RatecodeID: double (nullable = false)
 |-- TotalAmount: double (nullable = true)
 |-- PaymentType: integer (nullable = false)



## Abgeleitete Spalten erzeugen

In [35]:
yellow_taxi_df = (
    yellow_taxi_df
        .withColumn("TripYear", year(col("PickupTime")))   # withColum-Style
        .select(                                           # SQL-Style
            "*",
            expr("month(PickupTime) AS TripMonth"),        # SQL Funktionen aufrufen
            dayofmonth(col("PickupTime")).alias("TripDay") # Python Funktionen
        )
)
yellow_taxi_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- PassengerCount: integer (nullable = true)
 |-- TripDistance: double (nullable = true)
 |-- PickupTime: timestamp (nullable = true)
 |-- DropTime: timestamp (nullable = true)
 |-- PickupLocationId: integer (nullable = true)
 |-- DropLocationId: integer (nullable = true)
 |-- TotalAmount: double (nullable = true)
 |-- PaymentType: integer (nullable = false)
 |-- TripYear: integer (nullable = true)
 |-- TripMonth: integer (nullable = true)
 |-- TripDay: integer (nullable = true)
 |-- TripMonth: integer (nullable = true)
 |-- TripDay: integer (nullable = true)
 |-- TripTimeInMinutes: double (nullable = true)
 |-- NextTripTimeInMinutes: double (nullable = true)
 |-- TripType: string (nullable = false)
 |-- TripMonth: integer (nullable = true)
 |-- TripDay: integer (nullable = true)
 |-- TripMonth: integer (nullable = true)
 |-- TripDay: integer (nullable = true)



## Noch weitere Spalten berechnen lassen

In [24]:
# Trip Time in Minutes Option 1
yellow_taxi_df = (
    yellow_taxi_df
        .withColumn("TripTimeInMinutes",
                    round(
                        (unix_timestamp(col("DropTime"))-unix_timestamp(col("PickupTime")))/60 
                    )
                   )
)
yellow_taxi_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- PassengerCount: integer (nullable = true)
 |-- TripDistance: double (nullable = true)
 |-- PickupTime: timestamp (nullable = true)
 |-- DropTime: timestamp (nullable = true)
 |-- PickupLocationId: integer (nullable = true)
 |-- DropLocationId: integer (nullable = true)
 |-- RatecodeID: double (nullable = false)
 |-- TotalAmount: double (nullable = true)
 |-- PaymentType: integer (nullable = false)
 |-- TripYear: integer (nullable = true)
 |-- TripMonth: integer (nullable = true)
 |-- TripDay: integer (nullable = true)
 |-- TripMonth: integer (nullable = true)
 |-- TripDay: integer (nullable = true)
 |-- TripTimeInMinutes: double (nullable = true)



In [25]:
# wir können den code auch auslagern damit das wartbarer wird

trip_time_in_seconds_expr = unix_timestamp(col("DropTime")) - unix_timestamp(col("PickupTime"))

trip_time_in_minutes_expr = round(trip_time_in_seconds_expr / 60)


yellow_taxi_df = (
    yellow_taxi_df.withColumn("NextTripTimeInMinutes", trip_time_in_minutes_expr)
)
yellow_taxi_df.printSchema()
                                        

root
 |-- VendorID: integer (nullable = true)
 |-- PassengerCount: integer (nullable = true)
 |-- TripDistance: double (nullable = true)
 |-- PickupTime: timestamp (nullable = true)
 |-- DropTime: timestamp (nullable = true)
 |-- PickupLocationId: integer (nullable = true)
 |-- DropLocationId: integer (nullable = true)
 |-- RatecodeID: double (nullable = false)
 |-- TotalAmount: double (nullable = true)
 |-- PaymentType: integer (nullable = false)
 |-- TripYear: integer (nullable = true)
 |-- TripMonth: integer (nullable = true)
 |-- TripDay: integer (nullable = true)
 |-- TripMonth: integer (nullable = true)
 |-- TripDay: integer (nullable = true)
 |-- TripTimeInMinutes: double (nullable = true)
 |-- NextTripTimeInMinutes: double (nullable = true)



## Berechnete Werte zum Dritten

In [26]:
trip_type_column_expr = (
    when(
        col("RatecodeID") == 6, "SharedTrip"
    ).otherwise("SoloTrip")
)

yellow_taxi_df = yellow_taxi_df.withColumn("TripType", trip_type_column_expr).drop("RateCodeId")
yellow_taxi_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- PassengerCount: integer (nullable = true)
 |-- TripDistance: double (nullable = true)
 |-- PickupTime: timestamp (nullable = true)
 |-- DropTime: timestamp (nullable = true)
 |-- PickupLocationId: integer (nullable = true)
 |-- DropLocationId: integer (nullable = true)
 |-- TotalAmount: double (nullable = true)
 |-- PaymentType: integer (nullable = false)
 |-- TripYear: integer (nullable = true)
 |-- TripMonth: integer (nullable = true)
 |-- TripDay: integer (nullable = true)
 |-- TripMonth: integer (nullable = true)
 |-- TripDay: integer (nullable = true)
 |-- TripTimeInMinutes: double (nullable = true)
 |-- NextTripTimeInMinutes: double (nullable = true)
 |-- TripType: string (nullable = false)



### Ausführpläne

In [17]:
yellow_taxi_df.explain(mode="extended")

== Parsed Logical Plan ==
Project [VendorID#319, PassengerCount#496, TripDistance#484, PickupTime#507, DropTime#518, PickupLocationId#529, DropLocationId#540, TotalAmount#551, PaymentType#562, TripYear#573, TripMonth#585, TripDay#586, TripTimeInMinutes#600, NextTripTimeInMinutes#615, TripType#631]
+- Project [VendorID#319, PassengerCount#496, TripDistance#484, PickupTime#507, DropTime#518, PickupLocationId#529, DropLocationId#540, RatecodeID#420, TotalAmount#551, PaymentType#562, TripYear#573, TripMonth#585, TripDay#586, TripTimeInMinutes#600, NextTripTimeInMinutes#615, CASE WHEN (RatecodeID#420 = cast(6 as double)) THEN SharedTrip ELSE SoloTrip END AS TripType#631]
   +- Project [VendorID#319, PassengerCount#496, TripDistance#484, PickupTime#507, DropTime#518, PickupLocationId#529, DropLocationId#540, RatecodeID#420, TotalAmount#551, PaymentType#562, TripYear#573, TripMonth#585, TripDay#586, TripTimeInMinutes#600, round((cast((unix_timestamp(DropTime#518, yyyy-MM-dd HH:mm:ss, Some(Etc

## Zugriffe auf nested Json Content

In [29]:
import schemata

taxi_bases_df = (
    spark
        .read
        .option("multiline", True)
        .schema(schemata.taxi_bases_schema)
        .json("TaxiBases.json")
)
taxi_bases_df.show(truncate=False)

+--------------+--------------------------------------+----------------+------------+---------------------------+-----------------------------------------------------------+------------------------------------------------+
|License Number|Entity Name                           |Telephone Number|SHL Endorsed|Type of Base               |Address                                                    |GeoLocation                                     |
+--------------+--------------------------------------+----------------+------------+---------------------------+-----------------------------------------------------------+------------------------------------------------+
|B02865        |VIER-NY,LLC                           |6466657536      |No          |BLACK CAR BASE             |{636, WEST   28 STREET, NEW YORK, NY, 10001}               |{40.75273, -74.006408, (40.75273, -74.006408)}  |
|B02634        |VETERANS RADIO DISPATCHER CORP.       |7183647878      |No          |LIVERY BASE            

In [31]:
taxi_bases_flat_df = (
    taxi_bases_df.select(
        col("License Number").alias("BaseLicenseNumber"),
        col("Entity Name").alias("EntityName"),
        col("Address.Building").alias("AddressBuilding"),
        col("Address.Street").alias("AddressStreet"),
        col("Address.City").alias("AddressCity"),
        col("Address.State").alias("AddressState"),
        col("Address.Postcode").alias("AddressPostCode"),

        col("Geolocation.Latitude").alias("GeoLatitude"),
        col("Geolocation.Longitude").alias("GeoLongitude")
    )
)
taxi_bases_flat_df.show(truncate=False)

+-----------------+--------------------------------------+---------------+------------------------------+-------------+------------+---------------+-----------+------------+
|BaseLicenseNumber|EntityName                            |AddressBuilding|AddressStreet                 |AddressCity  |AddressState|AddressPostCode|GeoLatitude|GeoLongitude|
+-----------------+--------------------------------------+---------------+------------------------------+-------------+------------+---------------+-----------+------------+
|B02865           |VIER-NY,LLC                           |636            |WEST   28 STREET              |NEW YORK     |NY          |10001          |40.75273   |-74.006408  |
|B02634           |VETERANS RADIO DISPATCHER CORP.       |131            |KINGSBRIDGE ROAD              |BRONX        |NY          |10468          |40.86927   |-73.90281   |
|B80094           |ALPHA VAN LINE                        |115-54         |238 STREET                    |ELMONT       |NY         

## Aggregationen

In [32]:
yellow_taxi_report_df = (
    yellow_taxi_df
        .groupby("PickupLocationId", "DropLocationId")
        .agg(
            avg("TripTimeInMinutes").alias("AvgTripTime"),
            sum("TotalAmount").alias("SumAmount")
        )
        .orderBy(col("PickupLocationId").desc())
)
yellow_taxi_report_df.show()


+----------------+--------------+------------------+------------------+
|PickupLocationId|DropLocationId|       AvgTripTime|         SumAmount|
+----------------+--------------+------------------+------------------+
|             265|            16|              19.0|             32.76|
|             265|           230|              37.0|            383.34|
|             265|            86|23.666666666666668|              68.8|
|             265|           116|              40.0|             53.16|
|             265|           170|              40.0|350.43999999999994|
|             265|           231|              10.0|              15.3|
|             265|           218|              23.0|              45.0|
|             265|           233|              45.0|             78.85|
|             265|           132|              31.9|            598.71|
|             265|            23|               2.0|              41.6|
|             265|           144|              17.0|            

**Aufgaben**


* Neben einer Spalte für Durchschnitt basierend auf Trip-Time noch eine weitere für Max und Min einfügen.
* Basierend auf der Spalte PickupTime füge eine Spalte hinzu mit dem Namen des Monats hinzu.

[Doc zu strftime](https://www.programiz.com/python-programming/datetime/strftime)

[Api Doc PySpark Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#datetime-functions)